In [22]:
from sqlalchemy import create_engine
from sqlalchemy import text

import pandas as pd
import numpy as np
import psycopg2
from qiskit_aer import Aer
from qiskit.circuit.library import RealAmplitudes
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.quantum_info import SparsePauliOp

# **Database Configuration**
DB_HOST = "192.168.0.34"
DB_NAME = "rag_db"
DB_USER = "user"
DB_PASS = "password"
DB_PORT = "5432"

# **STEP 1: Connect to PostgreSQL using SQLAlchemy**
DATABASE_URL = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

def fetch_customer_data():
    """Fetches customer data from PostgreSQL using SQLAlchemy."""
    query = """
    SELECT 
        user_id, 
        current_credit_limit, 
        fico_score, 
        utilization_ratio, 
        annual_income 
    FROM customers
    """
    return pd.read_sql(query, engine)  # ✅ Fix: Now using SQLAlchemy engine

def quantum_credit_decision(fico, utilization, income):
    """Quantum optimization to determine credit limit increase."""
    num_qubits = 3  # Encoding FICO, Utilization, and Income as quantum states
    ansatz = RealAmplitudes(num_qubits, reps=2)

    # ✅ Fixed SparsePauliOp Formatting
    from qiskit.quantum_info import Pauli  
    H = SparsePauliOp(
        [Pauli("ZII"), Pauli("IZI"), Pauli("IIZ")], 
        coeffs=[fico / 850 * 0.4, -utilization / 100 * 0.3, income / 100000 * 0.3]
    )

    optimizer = COBYLA(maxiter=100)

    # ✅ Corrected VQE Initialization
    from qiskit.primitives import Estimator
    vqe = VQE(Estimator(), ansatz, optimizer=optimizer)

    result = vqe.compute_minimum_eigenvalue(H)

     # ✅ Print Debugging Values
    print(f"FICO: {fico}, Utilization: {utilization}, Income: {income} → Optimal Value: {result.optimal_value}")

    return np.clip(result.optimal_value, 1.0, 1.25)  # Credit increase between 1.0x - 1.25x




# **STEP 3: Apply Quantum Model to All Customers**
def update_credit_limits(df):
    df["increase_percentage"] = df.apply(
        lambda row: quantum_credit_decision(row["fico_score"], row["utilization_ratio"], row["annual_income"]), axis=1
    )
    
    # ✅ Print each customer's increase percentage for debugging
    for _, row in df.iterrows():
        print(f"User {row['user_id']} - Increase Factor: {row['increase_percentage']}x")

    df["new_credit_limit"] = df["current_credit_limit"] * df["increase_percentage"]
    return df


# **STEP 4: Update PostgreSQL with New Credit Limits**

def update_postgres_table(df):
    """Updates the PostgreSQL database with new credit limits using SQLAlchemy."""
    conn = engine.connect()
    trans = conn.begin()
    
    try:
        for _, row in df.iterrows():
            query = text("""
            UPDATE customers
            SET current_credit_limit = :credit_limit
            WHERE user_id = :user_id
            """)

            conn.execute(query, {"credit_limit": float(row["new_credit_limit"]), "user_id": int(row["user_id"])})
        
        trans.commit()
    except Exception as e:
        trans.rollback()
        print(f"Error updating database: {e}")
    finally:
        conn.close()


# **STEP 5: Run the Quantum Model**
if __name__ == "__main__":
    print("Fetching customer data from PostgreSQL...")
    customer_data = fetch_customer_data()
    
    print("Running quantum optimization for credit limits...")
    updated_data = update_credit_limits(customer_data)
    
    print("Updating new credit limits in PostgreSQL...")
    update_postgres_table(updated_data)
    
    print("Quantum-enhanced credit limit adjustments completed successfully!")


Fetching customer data from PostgreSQL...
Running quantum optimization for credit limits...


/tmp/ipykernel_2804865/3551060634.py:53: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe = VQE(Estimator(), ansatz, optimizer=optimizer)


FICO: 780.0, Utilization: 25.5, Income: 85000.0 → Optimal Value: -0.698558428804956


/tmp/ipykernel_2804865/3551060634.py:53: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe = VQE(Estimator(), ansatz, optimizer=optimizer)


FICO: 720.0, Utilization: 40.0, Income: 65000.0 → Optimal Value: -0.6538089661163201


...

FICO: 650.0, Utilization: 50.0, Income: 50000.0 → Optimal Value: -0.6058146472796254


/tmp/ipykernel_2804865/3551060634.py:53: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe = VQE(Estimator(), ansatz, optimizer=optimizer)


FICO: 600.0, Utilization: 60.0, Income: 45000.0 → Optimal Value: -0.5973503447022299


/tmp/ipykernel_2804865/3551060634.py:53: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe = VQE(Estimator(), ansatz, optimizer=optimizer)


FICO: 800.0, Utilization: 10.0, Income: 120000.0 → Optimal Value: -0.7664299847553514
User 1.0 - Increase Factor: 1.0x
User 2.0 - Increase Factor: 1.0x
User 3.0 - Increase Factor: 1.0x
User 4.0 - Increase Factor: 1.0x
User 5.0 - Increase Factor: 1.0x
Updating new credit limits in PostgreSQL...
Quantum-enhanced credit limit adjustments completed successfully!
